# Depencency parsing
(парсинг зависимостей)

## Что это?

* наша цель -- представить предложение естественного языка в виде дерева
* слова предложения -- вершины; *зависимости (dependencies)* между ними -- рёбра
* зависимости могут быть разными: например, субъект глагола, объект глагола, прилагательное-модификатор, и так далее

## Формат

Существует несколько форматов записи деревьев зависимостей, но самый популярный и общеиспользуемый -- [CoNLL-U](http://universaldependencies.org/format.html).<br/>
Как это выглядит (пример из [русского Universal Dependency трибанка](https://github.com/UniversalDependencies/UD_Russian-SynTagRus)):

In [1]:
my_example = """
# sent_id = 2003Armeniya.xml_138
# text = Перспективы развития сферы высоких технологий.
1	Перспективы	перспектива	NOUN	_	Animacy=Inan|Case=Nom|Gender=Fem|Number=Plur	0	ROOT	0:root	_
2	развития	развитие	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	1	nmod	1:nmod	_
3	сферы	сфера	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	2	nmod	2:nmod	_
4	высоких	высокий	ADJ	_	Case=Gen|Degree=Pos|Number=Plur	5	amod	5:amod	_
5	технологий	технология	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Plur	3	nmod	3:nmod	SpaceAfter=No
6	.	.	PUNCT	_	_	1	punct	1:punct	_
"""

Комментарии + таблица c 9 колонками (разделители табы):
* ID
* FORM: токен
* LEMMA: начальная форма
* UPOS: универсальная часть речи
* XPOS: лингво-специфичная часть речи
* FEATS: морфологическая информация: падеж, род, число etc
* HEAD: id ролителя
* DEPREL: тип зависимости, то есть отношение к токену-родителю
* DEPS: альтернативный подграф (не будем углубляться :))
* MISC: всё остальное

Отсутствующие данные представляются с помощью `_`. Больше подробностей про формат -- в [официальной документаци](http://universaldependencies.org/format.html).<br>


Отрытый инструмент для визуализации, ручной разметки и конвертации в другие форматы: UD Annotatrix. [Online-интерфейс](https://maryszmary.github.io/ud-annotatrix/standalone/annotator.html), [репозиторий](https://github.com/jonorthwash/ud-annotatrix).

Трибанк -- много таких предложений. Обычно они разделяются двумя переносами строки.
### Как считывать данные в питоне

Используем библиотеку [conllu](https://github.com/EmilStenstrom/conllu).

In [2]:
# !pip3 install conllu
from conllu import parse

In [3]:
help(parse)

Help on function parse in module conllu:

parse(data, fields=None, field_parsers=None)



In [4]:
my_example

'\n# sent_id = 2003Armeniya.xml_138\n# text = Перспективы развития сферы высоких технологий.\n1\tПерспективы\tперспектива\tNOUN\t_\tAnimacy=Inan|Case=Nom|Gender=Fem|Number=Plur\t0\tROOT\t0:root\t_\n2\tразвития\tразвитие\tNOUN\t_\tAnimacy=Inan|Case=Gen|Gender=Neut|Number=Sing\t1\tnmod\t1:nmod\t_\n3\tсферы\tсфера\tNOUN\t_\tAnimacy=Inan|Case=Gen|Gender=Fem|Number=Sing\t2\tnmod\t2:nmod\t_\n4\tвысоких\tвысокий\tADJ\t_\tCase=Gen|Degree=Pos|Number=Plur\t5\tamod\t5:amod\t_\n5\tтехнологий\tтехнология\tNOUN\t_\tAnimacy=Inan|Case=Gen|Gender=Fem|Number=Plur\t3\tnmod\t3:nmod\tSpaceAfter=No\n6\t.\t.\tPUNCT\t_\t_\t1\tpunct\t1:punct\t_\n'

In [5]:
sentences = parse(my_example)
sentence = sentences[0]
sentence[0]

OrderedDict([('id', 1),
             ('form', 'Перспективы'),
             ('lemma', 'перспектива'),
             ('upostag', 'NOUN'),
             ('xpostag', None),
             ('feats',
              OrderedDict([('Animacy', 'Inan'),
                           ('Case', 'Nom'),
                           ('Gender', 'Fem'),
                           ('Number', 'Plur')])),
             ('head', 0),
             ('deprel', 'ROOT'),
             ('deps', [('root', 0)]),
             ('misc', None)])

In [6]:
sentence[-1]

OrderedDict([('id', 6),
             ('form', '.'),
             ('lemma', '.'),
             ('upostag', 'PUNCT'),
             ('xpostag', None),
             ('feats', None),
             ('head', 1),
             ('deprel', 'punct'),
             ('deps', [('punct', 1)]),
             ('misc', None)])

## Визуализация

В nltk есть DependencyGraph, который умеет рисовать деревья (и ещё многое другое). Для того, чтобы визуализация работала корректно, ему нужна зависимость: graphviz.

```
sudo apt-get install graphviz
pip3 install graphviz
```

In [7]:
from nltk import DependencyGraph

В отличие от `conllu`, `DependencyGraph` не справляется с комментариями, поэтому придётся их убрать. Кроме того ему обязательно нужен `deprel` *ROOT* в верхнем регистре, иначе он не находит корень.

In [8]:
sents = []
for sent in my_example.split('\n\n'):
    # убираем коменты
    sent = '\n'.join([line for line in sent.split('\n') if not line.startswith('#')])
    # заменяем deprel для root
    sent = sent.replace('\troot\t', '\tROOT\t')
    sents.append(sent)

In [10]:
graph = DependencyGraph(tree_str=sents[0])
# graph

In [11]:
tree = graph.tree()
print(tree.pretty_print())

    Перспективы           
  _______|__________       
 |               развития 
 |                  |      
 |                сферы   
 |                  |      
 |              технологий
 |                  |      
 .               высоких  

None


## UDPipe

Есть разные инструменты для парсинга зависимостей. Сегодня мы будем рабтать с [UDPipe](http://ufal.mff.cuni.cz/udpipe). UDPipe умеет парсить текст с помощью готовых моделей (которые можно скачать [здесь](https://github.com/jwijffels/udpipe.models.ud.2.0/tree/master/inst/udpipe-ud-2.0-170801)) и обучать модели на своих трибанках.

Собственно, в UDPipe есть три вида моделей:
* токенизатор (разделить предложение не токены, сделать заготовку для CoNLL-U)
* тэггер (разметить части речи)
* сам парсер (проставить каждому токену `head` и `deprel`)

Мы сегодня не будем обучать новые модели (это слишком долго), а используем готовую модель для русского.


### Как работает UDPipe:

- Токенизация: текст разделяется на предложения, а предложения — на слова.
- Теггинг: по последним символам слова генерируются возможные POS-теги, далее нейросеть решает задачу классификации.
- Лемматизация и морфологический теггинг.
- Синтаксический парсинг.

![пример](https://habrastorage.org/getpro/habr/post_images/196/b17/845/196b17845e524d75a878837b25325a76.png)

### The Python binding

У udpipe есть питоновская обвязка. Она довольно [плохо задокументирована](https://pypi.org/project/ufal.udpipe/), но зато можно использовать прямо в питоне :)

In [12]:
from ufal.udpipe import Model, Pipeline

In [13]:
model = Model.load("russian-syntagrus-ud-2.0-170801.udpipe") # path to the model

In [14]:
# если успех, должно быть так (model != None)
model

<Swig Object of type 'model *' at 0x0000023184AEBAE8>

In [15]:
pipeline = Pipeline(model, 'generic_tokenizer', '', '', '')
example = "Если бы мне платили каждый раз. Каждый раз, когда я думаю о тебе."
parsed = pipeline.process(example)
print(parsed)

# newdoc
# newpar
# sent_id = 1
# text = Если бы мне платили каждый раз.
1	Если	если	SCONJ	_	_	4	mark	_	_
2	бы	бы	PART	_	Mood=Cnd	4	aux	_	_
3	мне	я	PRON	_	Case=Dat|Number=Sing|Person=1	4	obl	_	_
4	платили	платить	VERB	_	Aspect=Imp|Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
5	каждый	каждый	ADJ	_	Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing	6	amod	_	_
6	раз	раз	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	4	obl	_	SpaceAfter=No
7	.	.	PUNCT	_	_	6	punct	_	_

# sent_id = 2
# text = Каждый раз, когда я думаю о тебе.
1	Каждый	каждый	ADJ	_	Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing	2	amod	_	_
2	раз	раз	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	0	root	_	SpaceAfter=No
3	,	,	PUNCT	_	_	2	punct	_	_
4	когда	когда	SCONJ	_	_	2	mark	_	_
5	я	я	PRON	_	Case=Nom|Number=Sing|Person=1	6	nsubj	_	_
6	думаю	думать	VERB	_	Aspect=Imp|Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin|Voice=Act	2	advcl	_	_
7	о	о	ADP	_	_	8	case	_	_
8	тебе	ты	PRON

Как видим, UDPipe и токенизировал, и лематизировал текст, сделал POS-tagging и, собственно, синтаксический парсинг.

## Главред

Главред -- [сервис](https://glvrd.ru/) для корекции стиля текста. Кроме интерфейса, у него есть [API](https://glvrd.ru/api/)!<br>
На этом семинаре мы имплементируем несколько функций, делающих нечто похожее, на основе синтаксиса.

Возьмём простой пример: предложение, перегруженное однородными членами.

In [16]:
conj = 'Я пришла, включила компьютер, открыла почту, прочитала письмо, налила чай.'
print(pipeline.process(conj))

# newdoc
# newpar
# sent_id = 1
# text = Я пришла, включила компьютер, открыла почту, прочитала письмо, налила чай.
1	Я	я	PRON	_	Case=Nom|Number=Sing|Person=1	2	nsubj	_	_
2	пришла	приходить	VERB	_	Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	SpaceAfter=No
3	,	,	PUNCT	_	_	2	punct	_	_
4	включила	включать	VERB	_	Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	2	conj	_	_
5	компьютер	компьютер	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	4	obj	_	SpaceAfter=No
6	,	,	PUNCT	_	_	5	punct	_	_
7	открыла	открывать	VERB	_	Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	2	conj	_	_
8	почту	почта	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	7	obj	_	SpaceAfter=No
9	,	,	PUNCT	_	_	8	punct	_	_
10	прочитала	прочитывать	VERB	_	Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	2	conj	_	_
11	письмо	письмо	NOUN	_	Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing	10	obj	_	S

В таком случае очень много частей предложения, соединённых связью `conj` -- ей обозначаются однородные члены предложения. В таком случае можно просто посчитать количество таких связей:

In [17]:
import collections
def count_conj(sentence):
    deprel = collections.Counter()
    for i in range(len(parse(pipeline.process(sentence))[0])):
        deprel[parse(pipeline.process(sentence))[0][i]['deprel']] +=1
    return deprel['conj']
    # your code here

In [18]:
count_conj(conj)

4

Ещё одна проблема, на которую обращают внимание в Главреде -- [парцелляция](http://maximilyahov.ru/blog/all/parcel/). Часто она делает предложения менее читаемыми.

По словам Главреда,
> Признак парцелляции — предложение синтаксически неполное и само по себе не имеет смысла.

Например, в предложении нет субъекта:

In [19]:
ex = 'Выключил компьютер и заснул.'
print(pipeline.process(ex))

# newdoc
# newpar
# sent_id = 1
# text = Выключил компьютер и заснул.
1	Выключил	выключать	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
2	компьютер	компьютер	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	1	obj	_	_
3	и	и	CCONJ	_	_	4	cc	_	_
4	заснул	засыпать	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	1	conj	_	SpaceAfter=No
5	.	.	PUNCT	_	_	4	punct	_	SpacesAfter=\n




Или вообще что-то странное:

In [20]:
ex = 'А броско, шикарно и выделяля бы вас из толпы'
print(pipeline.process(ex))

# newdoc
# newpar
# sent_id = 1
# text = А броско, шикарно и выделяля бы вас из толпы
1	А	а	CCONJ	_	_	6	cc	_	_
2	броско	броско	ADV	_	Degree=Pos	6	parataxis	_	SpaceAfter=No
3	,	,	PUNCT	_	_	2	punct	_	_
4	шикарно	шикарно	ADV	_	Degree=Pos	6	advmod	_	_
5	и	и	PART	_	_	6	advmod	_	_
6	выделяля	выделялить	VERB	_	Aspect=Imp|Tense=Pres|VerbForm=Conv|Voice=Act	0	root	_	_
7	бы	бы	PART	_	Mood=Cnd	6	aux	_	_
8	вас	вы	PRON	_	Case=Acc|Number=Plur|Person=2	6	obj	_	_
9	из	из	ADP	_	_	10	case	_	_
10	толпы	толпа	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	6	obl	_	SpacesAfter=\n




Напишите функцию, которая помогает обнаруживать парцелляцию, опираясь на то, что предложение неполное:

In [21]:
def parcellation(sentence):
    deprel = collections.Counter()
    upostag = collections.Counter()
    reply = []
    for i in range(len(parse(pipeline.process(sentence))[0])):
        deprel[parse(pipeline.process(sentence))[0][i]['deprel']] +=1
        upostag[parse(pipeline.process(sentence))[0][i]['upostag']] +=1
    if deprel['nsubj'] == 0:
        reply.append("В предложении не хватает субъекта")
    if upostag['VERB'] == 0:
        reply.append("В предложении нет глагола")
    return reply

In [22]:
parcellation(ex)

['В предложении не хватает субъекта']

In [23]:
example = 'Какой красивый дом!'

In [24]:
parcellation(example)

['В предложении не хватает субъекта', 'В предложении нет глагола']

А теперь давайте усложним критерий на однородные члены: проверим, не состоит ли предложение из большого числа однородных предложений. Для этого проверим, являются ли однородные члены глаголами.

In [25]:
def parcellation(sentence):
    deprel = collections.Counter()
    upostag = collections.Counter()
    reply = []
    for i in range(len(parse(pipeline.process(sentence))[0])):
        deprel[parse(pipeline.process(sentence))[0][i]['deprel']] +=1
        upostag[parse(pipeline.process(sentence))[0][i]['upostag']] +=1
    if deprel['nsubj'] == 0:
        reply.append("В предложении не хватает субъекта")
    if upostag['VERB'] == 0:
        reply.append("В предложении нет глагола")
    if upostag['VERB'] > 2:
        reply.append("Предложение осложнено однородными членами(глаголами)")
    return reply

In [26]:
parcellation(conj)

['Предложение осложнено однородными членами(глаголами)']

Придумайте метрику для оценки качества предложений на основе написаных функций. Напишите функцию-оцениватель. Пусть она принимает на вход предложение, а возвращает оценку от 1 до 10.

In [27]:
def get_score(sentence):
    parcel = parcellation(sentence)
    if len(parcel) != 0:
        col = 3. # максимальное колличество синтаксических ошибок
        score = 10. * (1. - ((len(parcel))/col))
    else:
        score = 10.
    return score

'''Добавляя варианты ошибок в функцию parcellation, получим более плотную шкалу от 0 до 10'''

'Добавляя варианты ошибок в функцию parcellation, получим более плотную шкалу от 0 до 10'

In [28]:
conj

'Я пришла, включила компьютер, открыла почту, прочитала письмо, налила чай.'

In [29]:
get_score(conj)

6.666666666666668

In [30]:
get_score(example)

3.333333333333334

In [31]:
ex_2 = "Дед построил красивый дом"
get_score(ex_2)

10.0

Если осталось время, придумайте и напишите свою функцию, которая помогала бы оценивать качество текста. Добавьте её в метрику.

## SVO-triples

С помощью синтекстического парсинга можно извлекать из предложений тройки субъект-объект-глагол, которые можно использовать для извлечения информации из текста.  

In [32]:
sent = """1	Собянин	_	NOUN	_	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing|fPOS=NOUN++	2	nsubj	_	_
2	открыл	_	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act|fPOS=VERB++	0	ROOT	_	_
3	новый	_	ADJ	_	Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing|fPOS=ADJ++	4	amod	_	_
4	парк	_	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing|fPOS=NOUN++	2	dobj	_	_
5	и	_	CONJ	_	fPOS=CONJ++	4	cc	_	_
6	детскую	_	ADJ	_	Case=Acc|Degree=Pos|Gender=Fem|Number=Sing|fPOS=ADJ++	7	amod	_	_
7	площадку	_	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing|fPOS=NOUN++	4	conj	_	_
8	.	_	PUNCT	.	fPOS=PUNCT++.	2	punct	_	_"""

Тройки слово-слово-связь:

In [33]:
graph = DependencyGraph(tree_str=sent)
list(graph.triples())

[(('открыл', 'VERB'), 'nsubj', ('Собянин', 'NOUN')),
 (('открыл', 'VERB'), 'dobj', ('парк', 'NOUN')),
 (('парк', 'NOUN'), 'amod', ('новый', 'ADJ')),
 (('парк', 'NOUN'), 'cc', ('и', 'CONJ')),
 (('парк', 'NOUN'), 'conj', ('площадку', 'NOUN')),
 (('площадку', 'NOUN'), 'amod', ('детскую', 'ADJ')),
 (('открыл', 'VERB'), 'punct', ('.', 'PUNCT'))]

Тройки субъект-объект-глагол:

In [34]:
def get_sov(sent):
    graph = DependencyGraph(tree_str=sent)
    sov = {}
    for triple in graph.triples():
        if triple:
            if triple[0][1] == 'VERB':
                sov[triple[0][0]] = {'subj':'','obj':[]}
    for triple in graph.triples():
        if triple:
            if triple[1] == 'nsubj':
                if triple[0][1] == 'VERB':
                    sov[triple[0][0]]['subj']  = triple[2][0]
            if triple[1] == 'dobj':
                if triple[0][1] == 'VERB':
                    obj_1 = triple[2][0]
                    sov[triple[0][0]]['obj'].append(obj_1)
                    for triple_2 in graph.triples():
                        if triple_2:
                            if triple_2[0][0] == obj_1:
                                if triple_2[1] == 'conj':
                                    obj_2 = triple_2[2][0]
                                    sov[triple[0][0]]['obj'].append(obj_2)
    return sov

sov = get_sov(sent)
print(sov)

{'открыл': {'subj': 'Собянин', 'obj': ['парк', 'площадку']}}


Измените код выше так, чтобы учитывались:
    1. Однородные члены предложения 
        * (парк, площадка), (Германия, Щвейцария)
    2. Сложные сказуемые 
        * (начнет продавать), (запретил провозить)
    3. Непрямые объекты
        * (едет, Польшу), (спел, скандале)
        
 Выберите случайные 100 новостей из датасета lenta и извлеките из текстов новостей события в формате SOV-троек.